## Predição de Sucesso de Startups  

Notebook desenvolvido para explorar e modelar dados reais de startups, com foco em prever se uma empresa terá **sucesso (ativa/adquirida)** ou **insucesso (fechada)**.  
O projeto envolve análise exploratória, tratamento de valores ausentes, seleção de variáveis e aplicação de modelos de classificação binária, estimulando o uso de técnicas de pré-processamento e aprendizado de máquina aplicadas ao empreendedorismo.  

In [ ]:
# Importação de bibliotecas
import pandas as pd
import numpy as np
import sklearn as skl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

# Configurações de visualização
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Bibliotecas carregadas com sucesso")

### 1. Carregamento de dados

In [ ]:
# Carregamento do dataset principal
train_df = pd.read_csv("../data/train.csv")
test_df = pd.read_csv("../data/test.csv")

print("Dados de treino carregados com sucesso. Shape: {}".format(train_df.shape))
print("Dados de teste carregados com sucesso. Shape: {}".format(test_df.shape))

print("Primeiras 5 linhas dos dados de treino:")
print(train_df.head(5))

In [ ]:
print("Informações gerais sobre os treinos:")
train_df.info()

print("Estatísticas descritivas dos treinos:")
print(train_df.describe())

print("Valores ausentes nos treinos:")
print(train_df.isnull().sum()[train_df.isnull().sum() > 0])

print("Distribuição da variável alvo (labels):")
print(train_df["labels"].value_counts())
print(train_df["labels"].value_counts(normalize=True) * 100)

In [ ]:
# 3. Tratando dados ausentes
cols_to_impute_median = [
    'age_first_funding_year', 'age_last_funding_year',
    'age_first_milestone_year', 'age_last_milestone_year',
    'funding_total_usd'
]

for col in cols_to_impute_median:
    median_val = train_df[col].median()
    train_df[col].fillna(median_val, inplace=True)
    test_df[col].fillna(median_val, inplace=True)

print("Valores ausentes tratados pela mediana para colunas: {}.".format(cols_to_impute_median))
print("\nVerificando valores ausentes após imputação dos treinos:")
print(train_df[cols_to_impute_median].isnull().sum())


In [ ]:
# Aplicando one hot encoding para category_code
categorical_col = ["category_code"]

train_df = pd.get_dummies(train_df, columns=categorical_col, drop_first=True)
test_df = pd.get_dummies(test_df, columns=categorical_col, drop_first=True)

# Alinhamento entre colunas de treino e teste
missing_cols_in_test = set(train_df.columns) - set(test_df.columns)
missing_cols_in_train = set(test_df.columns) - set(train_df.columns)

for col in missing_cols_in_test:
    if col != 'labels':  #Evitando coluna alvo em teste
        test_df[col] = 0

for col in missing_cols_in_train:
    if col != 'labels':  #Evitando coluna alvo em treino
        train_df[col] = 0


In [ ]:
# Definição e separação de features e target e padronização

X_train = train_df.drop("labels", axis=1)
y_train = train_df["labels"]

X_test = test_df.copy()

numerical_cols = [
    col for col in X_train.columns
    if X_train[col].dtype in ["int64", "float64"]
    and not col.startswith("is_")
    and not col.startswith("category_code_")
]

scaler = StandardScaler()

# Evitar warning com .loc para atribuições diretas
X_train.loc[:, numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test.loc[:, numerical_cols] = scaler.transform(X_test[numerical_cols])

print("Features e target separados. Colunas numéricas padronizadas.")
print("Primeiras 5 linhas de X_train após padronização:")
print(X_train.head())

In [ ]:
# Separando do conjuntos de treinos em treino e validação
X_train_full = train_df.drop("labels", axis=1)
y_train_full = train_df["labels"]

X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full
)

In [ ]:
# Aplicando padronização
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_val[numerical_cols] = scaler.transform(X_val[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

In [ ]:
# Definição para os hiperparâmetros C 
param_distributions = {'C': np.logspace(-3, 3, 100)}  # valores entre 0.001 e 1000 em escala log

logreg = LogisticRegression(random_state=42, solver='liblinear')

random_search = RandomizedSearchCV(
    logreg,
    param_distributions,
    n_iter=20,          # número de amostras a testar
    cv=5,               # folds da validação cruzada
    scoring='accuracy',  # métrica para avaliar
    n_jobs=-1,
    random_state=42,
    verbose=1
)

In [ ]:
# Ajuste do conjunto de treino
random_search.fit(X_train, y_train)

print(f"Melhor valor de C encontrado: {random_search.best_params_['C']}")

best_model = random_search.best_estimator_

# Avaliação no conjunto de validação
y_val_pred = best_model.predict(X_val)

print("Relatório de classificação no conjunto de validação:")
print(classification_report(y_val, y_val_pred))
print("Acurácia no conjunto de validação: {:.2f}".format(accuracy_score(y_val, y_val_pred)))

In [ ]:
# Alinhar colunas para evitar erro no predict
X_test = X_test[X_train.columns]

# Previsão no conjunto de teste
predictions = best_model.predict(X_test)

In [ ]:
# Geração do arquivo de submissão
submission_df = pd.DataFrame({
    "id": test_df["id"],
    "labels": predictions
})

submission_df.to_csv("../data/submission.csv", index=False)

print("Arquivo de submissão gerado com sucesso: ../data/submission.csv")